**Занятие первое**

Начнем с простого. Многие знают что такое map и reduce операции, но все же для закрпеления мы их тут реализуем. Ах да, не забудем и про shuffle. Делать все будем на упрощенной задаче с word count для ознакомления с самим подходом.

На самом деле мы рассмптрим все в упрощенном виде, но это даст нам понимание, как можно через hadoop streaming, например, писать самописные map и reduce операции

! mapred streaming \
  -input /wiki/sample.jsonl \
  -output /word-count \
  -mapper "/opt/conda/bin/python3.6 mapper.py" \
  -reducer "/opt/conda/bin/python3.6 reducer.py" \
  -file mapper.py \
  -file reducer.py

Выше mapper.py и reducer.py это программы, которые выполняют одноименные операции нам потоком информации из jsonl файла, записывая ответ в файл word-count

In [157]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
from nltk.corpus import stopwords
import re 
import string

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\denis\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\denis\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\denis\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Давайте загрузим файл с текстом и посмотрим на него

In [158]:
with open('spark_text.txt', 'rb') as f:
    data = f.readlines()
data = [text.decode() for text in data if text.decode() != '\r\n']    

In [159]:
len(data)

60

In [160]:
data[1]

'Apache Spark has its architectural foundation in the resilient distributed dataset (RDD), a read-only multiset of data items distributed over a cluster of machines, that is maintained in a fault-tolerant way.[2] The Dataframe API was released as an abstraction on top of the RDD, followed by the Dataset API. In Spark 1.x, the RDD was the primary application programming interface (API), but as of Spark 2.x use of the Dataset API is encouraged[3] even though the RDD API is not deprecated.[4][5] The RDD technology still underlies the Dataset API.[6][7]\r\n'

Как бы мы сделали..
Надо немного почистить слова, а также сделать все в парадигме MapReduce. Понятно, что можно все написать проще, но мы ведь хотим понять, как это работает=)

Загрузим стоп слова, очистим от них текст, приведем к нижнему регистру, всем раздадим ключи

In [161]:
stop_words = stopwords.words("english")
stop_words = set(stop_words)

In [162]:
stop_words

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

пунктуацию тоже полезно бы удалить

In [163]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [164]:
def mapper_text(text):
    clean_text = re.sub(rf"[{string.punctuation}]", "", text)
    words = nltk.word_tokenize(clean_text)
    words_with_value = [(word.lower(), 1) for word in words 
                        if word not in stop_words]
    words_with_value = sorted(words_with_value, key=lambda x:x[0])
    return words_with_value

def create_chunks(shuffled_data):
    result = {}
    for idx, data in shuffled_data:
        if idx in result:
            result[idx].append(data)
        else:
            result[idx] = [data]
    return list(result.items())

def shuffle_text(mapper_result, n_nodes=5):
    shuffled_data = []
    for key, value in mapper_result:
        shuffled_data.append((hash(key)%n_nodes, (key, value)))
    shuffled_data = sorted(shuffled_data, key=lambda x: x[0])
    chunks = create_chunks(shuffled_data)
    return chunks


# на самом деле для reduce в жизни пишут иначе..не зря мы сортируем внутри map
#данные по ключам. Это нужно для избавления от этапа проверки ключа и поиска
def reduce_text(values_to_reduce):
    result = {}
    for key, value in values_to_reduce:
        if key in result:
            result[key] += 1
        else:
            result[key] = 1
    return result

Проверим, что все работает

Сначала map

In [165]:
data[0]

'Apache Spark is an open-source unified analytics engine for large-scale data processing. Spark provides an interface for programming entire clusters with implicit data parallelism and fault tolerance. Originally developed at the University of California, Berkeley AMPLab, the Spark codebase was later donated to the Apache Software Foundation, which has maintained it since.\r\n'

In [166]:
map_stage = mapper_text(data[0])

In [167]:
map_stage

[('amplab', 1),
 ('analytics', 1),
 ('apache', 1),
 ('apache', 1),
 ('berkeley', 1),
 ('california', 1),
 ('clusters', 1),
 ('codebase', 1),
 ('data', 1),
 ('data', 1),
 ('developed', 1),
 ('donated', 1),
 ('engine', 1),
 ('entire', 1),
 ('fault', 1),
 ('foundation', 1),
 ('implicit', 1),
 ('interface', 1),
 ('largescale', 1),
 ('later', 1),
 ('maintained', 1),
 ('opensource', 1),
 ('originally', 1),
 ('parallelism', 1),
 ('processing', 1),
 ('programming', 1),
 ('provides', 1),
 ('since', 1),
 ('software', 1),
 ('spark', 1),
 ('spark', 1),
 ('spark', 1),
 ('tolerance', 1),
 ('unified', 1),
 ('university', 1)]

shuffle

In [168]:
shuffle_stage = shuffle_text(map_stage, 5)

In [169]:
shuffle_stage

[(0,
  [('codebase', 1),
   ('donated', 1),
   ('implicit', 1),
   ('maintained', 1),
   ('opensource', 1),
   ('provides', 1),
   ('university', 1)]),
 (1,
  [('engine', 1),
   ('fault', 1),
   ('foundation', 1),
   ('largescale', 1),
   ('parallelism', 1),
   ('since', 1),
   ('spark', 1),
   ('spark', 1),
   ('spark', 1),
   ('unified', 1)]),
 (2,
  [('amplab', 1),
   ('analytics', 1),
   ('clusters', 1),
   ('developed', 1),
   ('entire', 1),
   ('later', 1),
   ('originally', 1),
   ('processing', 1),
   ('software', 1)]),
 (3, [('data', 1), ('data', 1), ('programming', 1)]),
 (4,
  [('apache', 1),
   ('apache', 1),
   ('berkeley', 1),
   ('california', 1),
   ('interface', 1),
   ('tolerance', 1)])]

reduce

In [170]:
reduce_text(shuffle_stage[4][1])

{'apache': 2, 'berkeley': 1, 'california': 1, 'interface': 1, 'tolerance': 1}

Итак, осталось все рассчитать параллельно и собрать результаты

In [171]:
from joblib import Parallel, delayed

In [172]:
n_nodes = 5

Обернем в 1 функциию для удобства map и shuffle

In [173]:
def map_shuffle(text, n_nodes):
    map_result = mapper_text(text)
    shuffle_result = shuffle_text(map_result, n_nodes)
    return shuffle_result

In [174]:
with Parallel(n_jobs=n_nodes, verbose=10, batch_size=5) as parallel:
    res = parallel(delayed(map_shuffle)(df, n_nodes) for df in data)

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   6 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done  25 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done  60 out of  60 | elapsed:    0.0s finished


In [175]:
len(res)

60

In [176]:
res[0]

[(0,
  [('analytics', 1),
   ('clusters', 1),
   ('codebase', 1),
   ('implicit', 1),
   ('later', 1),
   ('opensource', 1),
   ('since', 1),
   ('university', 1)]),
 (1,
  [('california', 1),
   ('data', 1),
   ('data', 1),
   ('engine', 1),
   ('entire', 1),
   ('maintained', 1)]),
 (2,
  [('developed', 1), ('donated', 1), ('largescale', 1), ('parallelism', 1)]),
 (3,
  [('amplab', 1),
   ('apache', 1),
   ('apache', 1),
   ('berkeley', 1),
   ('fault', 1),
   ('originally', 1),
   ('processing', 1),
   ('provides', 1),
   ('unified', 1)]),
 (4,
  [('foundation', 1),
   ('interface', 1),
   ('programming', 1),
   ('software', 1),
   ('spark', 1),
   ('spark', 1),
   ('spark', 1),
   ('tolerance', 1)])]

Сделаем что-то вроде перессылки, собирая все в словари и заодно посмотрим на сколько равномерно распределлиись наши слова

In [177]:
shuffle_stage = {i:[] for i in range(5)}
for values in res:
    values = dict(values)
    for key in values.keys():
        shuffle_stage[key].extend(values[key])

In [178]:
for key in shuffle_stage.keys():
    print(f'{key}: number of words = {len(shuffle_stage[key])}')

0: number of words = 364
1: number of words = 393
2: number of words = 352
3: number of words = 406
4: number of words = 406


И последний этап - нужно сделать reduce

In [179]:
with Parallel(n_jobs=n_nodes, verbose=10, batch_size=5) as parallel:
    res = parallel(delayed(reduce_text)(shuffle_stage[key]) for key in shuffle_stage.keys())

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=5)]: Done   3 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:    0.0s finished


In [180]:
len(res)

5

In [181]:
res[0]

{'analytics': 5,
 'clusters': 1,
 'codebase': 1,
 'implicit': 1,
 'later': 1,
 'opensource': 1,
 'since': 1,
 'university': 1,
 '1x': 1,
 'dataframe': 1,
 'even': 1,
 'items': 2,
 'computing': 1,
 'function': 2,
 'limitations': 1,
 'particular': 1,
 'spark': 21,
 'store': 1,
 'memory8': 1,
 'restricted': 2,
 'implementation': 5,
 'multiple': 2,
 'learning': 7,
 'machine': 5,
 'magnitude': 1,
 'training': 1,
 'also': 4,
 'manually': 1,
 'native': 1,
 'run': 6,
 'custom': 3,
 'kudu': 1,
 'single': 1,
 'usually': 1,
 'variety': 2,
 'application': 3,
 'java': 2,
 'languages': 2,
 'nonjvm': 1,
 'input': 1,
 'ones': 2,
 'parallel': 4,
 'scala': 4,
 'sequence': 2,
 'available': 1,
 'besides': 2,
 'broadcast': 1,
 'forms': 1,
 'functional': 2,
 'style': 1,
 'argument': 1,
 'each': 1,
 'following': 1,
 'item': 1,
 'occurring': 1,
 'set': 2,
 'called': 2,
 'commandline': 1,
 'compiletime': 1,
 'core': 3,
 'dsl': 1,
 'manipulate': 1,
 'net16': 1,
 'well': 3,
 'minibatches': 3,
 'thus': 1,
 'built

Собираем результат

In [182]:
result = {}
for partition in res:
    for key in partition.keys():
        if key in result:
            result[key] += partition[key]
        else:
            result[key] = partition[key]

In [183]:
sorted(result.items(), key=lambda x: x[1], reverse=True)

[('spark', 110),
 ('apache', 56),
 ('data', 44),
 ('streaming', 36),
 ('distributed', 23),
 ('processing', 19),
 ('sql', 17),
 ('learning', 15),
 ('also', 15),
 ('rdd', 15),
 ('api', 14),
 ('structured', 13),
 ('machine', 12),
 ('cluster', 12),
 ('hadoop', 11),
 ('interface', 10),
 ('provides', 10),
 ('rdds', 10),
 ('mapreduce', 10),
 ('core', 9),
 ('framework', 9),
 ('use', 9),
 ('graph', 9),
 ('the', 9),
 ('mllib', 9),
 ('programming', 9),
 ('application', 8),
 ('python', 8),
 ('used', 8),
 ('code', 8),
 ('support', 8),
 ('tasks', 8),
 ('run', 7),
 ('operations', 7),
 ('deep', 7),
 ('it', 7),
 ('algorithms', 7),
 ('in', 7),
 ('dataset', 7),
 ('graphx', 7),
 ('implementation', 6),
 ('scala', 6),
 ('developers', 6),
 ('including', 6),
 ('simple', 6),
 ('pipelines', 6),
 ('batch', 6),
 ('abstraction', 6),
 ('map', 6),
 ('two', 6),
 ('big', 6),
 ('graphs', 6),
 ('analytics', 5),
 ('even', 5),
 ('computing', 5),
 ('java', 5),
 ('parallel', 5),
 ('set', 5),
 ('applications', 5),
 ('using',

Да, было бы проще все сделать иным кодом и в один проход, но целью было разобрать, как все это примерно работает под капотом на больших данных.

**Домашнее задание**

Посчитать количество рейтингов больше 4 для каждого фильма и вывести фильмы в порядке убывания количества этих оценок

In [184]:
with open('user_ratedmovies.dat', 'rb') as f:
    data = f.readlines()
headers = data[0].decode().split('\t')[:3]
data = [row.decode().split('\t')[:3] for row in data[1:]]   

In [185]:
headers

['userID', 'movieID', 'rating']

In [186]:
data[0]

['75', '3', '1']

In [187]:
len(data)

855598

Пишем map, shiffle и reduce + параллелим вычисления. Лучше задавать batch_size при распараллеливании, либо даже заранее все разбить на батчи, будет быстрее

Также посмотрите на то, нет ли перекоса в данных после shuffle, можете попробовать использовать остаток от деления не простого hash, а ввести какую-то функию

In [188]:
# Зададим хэш-функцию с меньшим перекосом данных после shuffle

import hashlib

def new_hash(obj_to_hash: int):
    encoded_obj = str(obj_to_hash).encode()
    hash_16 = hashlib.md5(encoded_obj).hexdigest()
    return int(hash_16 ,16)


In [189]:
def map_rating(row):
    films_with_rating = [(int(row[1]), float(row[2]))]
    return films_with_rating

def create_chunks(shuffled_data):
    result = {}
    for idx, data in shuffled_data:
        if idx in result:
            result[idx].append(data)
        else:
            result[idx] = [data]
    return list(result.items())

def shuffle_rating(mapper_result, n_nodes=5):
    shuffled_data = []
    for key, value in mapper_result:
        if value > 4:
            shuffled_data.append((new_hash(key)%n_nodes, (key, value)))
    shuffled_data = sorted(shuffled_data, key=lambda x: x[0])
    chunks = create_chunks(shuffled_data)
    return chunks

def reduce_rating(map_row):
    result = {}
    for key, value in map_row:
        if key in result:
            result[key] += 1
        elif key not in result:
            result[key] = 1
    return result

def map_shuffle(row, n_nodes):
    map_result = map_rating(row)
    shuffle_result = shuffle_rating(map_result, n_nodes)
    return shuffle_result

In [190]:
with Parallel(n_jobs=n_nodes, verbose=10, batch_size=1000) as parallel:
    res = parallel(delayed(map_shuffle)(row, n_nodes) for row in data)

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   6 tasks      | elapsed:    0.0s
[Parallel(n_jobs=5)]: Done 3010 tasks      | elapsed:    0.1s
[Parallel(n_jobs=5)]: Done 8010 tasks      | elapsed:    0.1s
[Parallel(n_jobs=5)]: Done 15010 tasks      | elapsed:    0.2s
[Parallel(n_jobs=5)]: Done 22010 tasks      | elapsed:    0.2s
[Parallel(n_jobs=5)]: Done 31010 tasks      | elapsed:    0.2s
[Parallel(n_jobs=5)]: Done 40010 tasks      | elapsed:    0.3s
[Parallel(n_jobs=5)]: Done 51010 tasks      | elapsed:    0.3s
[Parallel(n_jobs=5)]: Done 62010 tasks      | elapsed:    0.4s
[Parallel(n_jobs=5)]: Done 75010 tasks      | elapsed:    0.4s
[Parallel(n_jobs=5)]: Done 88010 tasks      | elapsed:    0.6s
[Parallel(n_jobs=5)]: Done 103010 tasks      | elapsed:    0.7s
[Parallel(n_jobs=5)]: Done 118010 tasks      | elapsed:    0.7s
[Parallel(n_jobs=5)]: Done 135010 tasks      | elapsed:    0.8s
[Parallel(n_jobs=5)]: Done 152010 tasks     

In [191]:
shuffle_stage = {i:[] for i in range(5)}
for values in res:
    values = dict(values)
    for key in values.keys():
        shuffle_stage[key].extend(values[key])

In [192]:
for key in shuffle_stage.keys():
    print(f'{key}: number of words = {len(shuffle_stage[key])}')

0: number of words = 32524
1: number of words = 30107
2: number of words = 32131
3: number of words = 30973
4: number of words = 34597


In [193]:
with Parallel(n_jobs=n_nodes, verbose=10, batch_size=5) as parallel:
    res = parallel(delayed(reduce_rating)(shuffle_stage[key]) for key in shuffle_stage.keys())

[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   2 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=5)]: Done   3 out of   5 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=5)]: Done   5 out of   5 | elapsed:    0.0s finished


In [194]:
result = {}
for partition in res:
    for key in partition.keys():
        if key in result:
            result[key] += partition[key]
        else:
            result[key] = partition[key]

In [195]:
output = []

for x in sorted(result.items(), key=lambda x: x[1], reverse=True)[:10]:
    output.append([x[0], x[1]])
    print(x[0], x[1])

# output

2571 900
318 887
296 878
2959 828
4993 756
7153 719
5952 697
858 690
50 688
2858 680


После reduce все можно собрать в одном цикле как на семинаре

В качестве ответа вывести топ 10 фильмов с наибольшим числом оценок более 4

### Срок выполнения домашего задания - 23.09.2024